In [4]:
# pip install sentence-transformers

     ---------------------------------------- 0.0/268.7 kB ? eta -:--:--
     --------------- ---------------------- 112.6/268.7 kB 2.2 MB/s eta 0:00:01
     -------------------------------------  266.2/268.7 kB 4.1 MB/s eta 0:00:01
     -------------------------------------- 268.7/268.7 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
     - -------------------------------------- 0.5/10.0 MB 9.8 MB/s eta 0:00:01
     --- ------------------------------------ 0.8/10.0 MB 7.6 MB/s eta 0:00:02
     ---- ----------------------------------- 1.1/10.0 MB 6.2 MB/s eta 0:00:02
     ----- ---------------------------------- 1.4/10.0 MB 6.0 MB/s eta 0:00:02
     ------ --------------------------------- 1.8/10.0 MB 6.2 MB/s eta 0:00:02
     -------- ------------------------------- 2.2/10.0 MB 6.3 MB/s eta 0:00:02
     ---------- ----------------------------- 2.5/10.0 MB 6.2 MB/s eta 0:00:02
     ----------- ---------------------------- 2.8/10.0 MB 6.2 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
import faiss
import re, os, openai, glob
from bs4 import BeautifulSoup as Soup
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.callbacks import get_openai_callback
from decouple import config

In [2]:
print(openai.__version__)

1.43.0


In [53]:
pdf_path = "D:\gen_ai_bot\Service_Renewal of family residence for GCC families.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()


In [54]:
documents[0].page_content

'Service:\nRenewal\nof\nfamily\nresidence\nFor\nGCC\nCountries\nCitizen\nFor\nthe\nRenewal\nof\nfamily\nresidence\nthere\ncan\nbe\n3\ntypes\nof\nguarantor:\n●\nCitizen\n●\nGCC\ncountries\ncitizen\n●\nForeign\ncitizen\nDepending\non\nthe\nguarantor\nand\ntheir\nfamily\nmembers\nthe\ndocuments\nrequired\nfor\nthis\nresidency\nrenewal\nservice\nwould\nbe\ndifferent.\nFor\nGCC\nCountry\nCitizen\nIf\nthe\nmember\nis\nborn\nin\nthis\ncountry ,\nthey\nhave\nto\nsubmit\nthe\nfollowing\ndocuments\n:\n1.\nA\nrecent\ncolored\npersonal\nphoto\nwith\na\nwhite\nbackground\nof\nthe\nsponsored\nperson.\n2.\nA\ncopy\nof\nthe\npassport.\n3.\nCertified\nbirth\ncertificate.\nIf\nthe\nmember\nis\ntheir\nchild\nand\nunder\n18\nyears,\nthe\ndocuments\nwould\ninclude:\n1.\nA\nrecent\ncolored\npersonal\nphoto\nwith\na\nwhite\nbackground\nof\nthe\nsponsored\nperson.\n2.\nA\ncopy\nof\nthe\npassport.\nIf\nthe\nmember\nis\ntheir\nchild\nabove\nthe\nage\nof\n18,\nthe\nfollowing\ndocuments\nwould\nneed\nto\nbe\nsubm

In [55]:
# Access the first document
first_document = documents[1]
print(first_document.page_content)

Service
Steps
●
Through
Digital
channels
(website/smart
application):
1.
Log
in
to
the
smart
services
system
through
(UAE
Pass
or
username).
2.
Search
for
the
service
to
be
applied
for.
3.
Fill
in
the
application
data,
where
applicable.
4.
Pay
the
service
fee
(if
any).
●
Through
Customer
Happiness
Center:
1.
Visit
the
nearest
Customer
Happiness
Center.
2.
Get
the
automated
turn
ticket
and
wait.
3.
Submitting
the
application
that
fulfils
all
conditions
and
documents
(if
any)
to
the
customer
service
employee.
4.
Pay
the
service
fee
(if
any).
●
Through
Amer
Service
Center:
1.
Visit
the
nearest
Amer
Service
Center.
2.
Get
the
automated
turn
ticket
and
wait.
3.
Submitting
the
application
that
fulfils
all
conditions
and
documents
(if
any)
to
the
customer
service
employee.
4.
Pay
the
service
fee
(if
any).
Service
Availability
Channels
●
Digital
channels
(website/smart
application):
The
service
is
available
24/7.
●
Amer
Service
Centre:
The
service
is
available
during
the
normal
operating
hours

In [56]:
for i in range(len(documents)):
    documents[i].page_content = "this data is extracted from PDF.\n " + documents[i].page_content 

In [57]:
openai_key = config("OPENAI_API_KEY")
openai.api_key = openai_key
os.environ['OPENAI_API_KEY'] = openai_key

In [58]:
len(documents)

2

In [59]:
documents

[Document(page_content='this data is extracted from PDF.\n Service:\nRenewal\nof\nfamily\nresidence\nFor\nGCC\nCountries\nCitizen\nFor\nthe\nRenewal\nof\nfamily\nresidence\nthere\ncan\nbe\n3\ntypes\nof\nguarantor:\n●\nCitizen\n●\nGCC\ncountries\ncitizen\n●\nForeign\ncitizen\nDepending\non\nthe\nguarantor\nand\ntheir\nfamily\nmembers\nthe\ndocuments\nrequired\nfor\nthis\nresidency\nrenewal\nservice\nwould\nbe\ndifferent.\nFor\nGCC\nCountry\nCitizen\nIf\nthe\nmember\nis\nborn\nin\nthis\ncountry ,\nthey\nhave\nto\nsubmit\nthe\nfollowing\ndocuments\n:\n1.\nA\nrecent\ncolored\npersonal\nphoto\nwith\na\nwhite\nbackground\nof\nthe\nsponsored\nperson.\n2.\nA\ncopy\nof\nthe\npassport.\n3.\nCertified\nbirth\ncertificate.\nIf\nthe\nmember\nis\ntheir\nchild\nand\nunder\n18\nyears,\nthe\ndocuments\nwould\ninclude:\n1.\nA\nrecent\ncolored\npersonal\nphoto\nwith\na\nwhite\nbackground\nof\nthe\nsponsored\nperson.\n2.\nA\ncopy\nof\nthe\npassport.\nIf\nthe\nmember\nis\ntheir\nchild\nabove\nthe\nage\nof\

In [60]:
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100)
# text_chunks=text_splitter.split_documents(documents)

In [61]:
embeddings=OpenAIEmbeddings()

In [62]:
vectorstore=FAISS.from_documents(documents,embeddings)

In [67]:
vectorstore.save_local('new_indexes/Renewal_of_family_residence_for_GCC_families')

In [14]:
vectorstore=FAISS.load_local('Status_amendment',embeddings)

In [15]:
# get vector from urldata
vectorstore2 = FAISS.load_local('status_edit', embeddings)

In [68]:
# v1 = FAISS.load_local('Residency_renewal_citizen_family_members', embeddings)
# v2 = FAISS.load_local('Residency_renewal_foreign_members', embeddings)
# v3 = FAISS.load_local('Residency_renewal_foreigner_family_members', embeddings)
# v4 = FAISS.load_local('Residency_renewal_gcc_family_members', embeddings)
# v5 = FAISS.load_local('Residency_visa_renewal_citizen_gcc_family', embeddings)
# v6 = FAISS.load_local('Status_amendment', embeddings)
# v7 = FAISS.load_local('status_edit', embeddings)

v1 = FAISS.load_local('new_indexes/Renewal_of_family_residence_for_citizen', embeddings)
v2 = FAISS.load_local('new_indexes/Renewal_of_family_residence_for_GCC_families', embeddings)
v3 = FAISS.load_local('new_indexes/Renewal_of_family_residence_of_foreigners', embeddings)
v4 = FAISS.load_local('new_indexes/Status_amendment', embeddings)


In [69]:
v1.merge_from(v2)
v1.merge_from(v3)
v1.merge_from(v4)

v1.save_local('new_indexes/merged_index')

In [182]:
vectorstore = FAISS.load_local("new_indexes/merged_index", embeddings)

In [22]:
# vectorstore.merge_from(vectorstore2)

NameError: name 'vectorstore2' is not defined

In [183]:
vectorstore.docstore._dict

{'1e6bf3bf-c08a-4e07-9780-8d2c050e9de6': Document(page_content='this data is extracted from PDF.\n Service:\nRenewal\nof\nfamily\nresidence\nFor\nCitizen\nFor\nthe\nRenewal\nof\nfamily\nresidence\nthere\ncan\nbe\n3\ntypes\nof\nguarantor:\n●\nCitizen\n●\nGCC\ncountries\ncitizen\n●\nForeign\ncitizen\nDepending\non\nthe\nguarantor\nand\ntheir\nfamily\nmembers\nthe\ndocuments\nrequired\nfor\nthis\nresidency\nrenewal\nservice\nwould\nbe\ndifferent.\nFor\nCitizen\nIf\nthe\ncitizen\nis\nmale,\nhe\nhas\nto\nsubmit\ndocuments\nfor\nhis\nwife\nand\nkids(if\nany).\nThe\ndocuments\nwould\ninclude:\n1.\nA\nrecent\ncolored\npersonal\nphoto\nwith\na\nwhite\nbackground\nof\nthe\nsponsored\nperson.\n2.\nA\ncopy\nof\nthe\npassport.\n3.\nA\nmedical\nexamination\napproved\nby\nthe\nauthorities.\nIf\nthe\ncitizen\nis\nfemale,\nshe\nhas\nto\nsubmit\ndocuments\nfor\nher\nhusband\nand\nstep-children(if\nany).\nThe\ndocuments\nwould\ninclude:\n1.\nA\nrecent\ncolored\npersonal\nphoto\nwith\na\nwhite\nbackground

In [184]:
from langchain import PromptTemplate
template = '''

If you don't know the answer, just say that you don't know.
Don't try to make up an answer.
First look at the data extracted from the pdf, if the answer was not found in the pdf data only then look at the data gathered from the URL.
Try to give complete information on the question. Summarize if you have to. 
Dont mention the source link.

Use the context (delimited by <ctx></ctx>) and chat history (delimited by <hs></hs>) to see if any relevant information is there and use that to answer the query.
<ctx>
{context}
context: You are a useful bot who can provide information about the products and services offered by the General Directorate of Residency and Foreigners Affairs - Dubai.


</ctx>

<hs>
{history}
</hs>

Question: {question}
Answer:
'''
prompt = PromptTemplate(
    template=template,
    input_variables=['history', 'context', 'question']
)

In [185]:
llm=ChatOpenAI(model_name='gpt-4-turbo')

In [186]:
retriever=vectorstore.as_retriever() #(search_kwargs={'k':6})search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7}
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt,
        "memory": ConversationBufferWindowMemory(
            k=3,
            memory_key="history",
            input_key="question"),
    }
)

In [188]:
with get_openai_callback() as cb:
  d_response=chain.invoke({"query":"time taken ?"})
  print(d_response["result"])
  print("Sources:")
  sources_set = set()
  for source_document in d_response["source_documents"]:
    sources_set.add(source_document.metadata['source'])
  for source in sources_set:
    print(source)
  print(cb)

The expected completion time for the service is 48 hours.
Sources:
D:/gen_ai_bot/Service_Renewal of family residence for GCC families.pdf
D:/gen_ai_bot/Service_Edit status or Status Amendment.pdf
D:/gen_ai_bot/Service_Renewal of family residence for citizen.pdf
D:/gen_ai_bot/Service_Renewal of family residence of foreigners.pdf
Tokens Used: 1716
	Prompt Tokens: 1704
	Completion Tokens: 12
Successful Requests: 1
Total Cost (USD): $0.0


In [153]:
d_response

{'query': 'what are the required documents for status amendment for a citizen ?',
 'result': "For a citizen requiring a status amendment, the necessary documents include:\n\n1. A copy of the passport of both the sponsored person and the sponsor.\n2. A copy of the sponsor's residence permit.\n3. Copies of either the cancelled entry permit or residence permit, along with the new entry permit.",
 'source_documents': [Document(page_content='this data is extracted from PDF.\n Service:\nEdit\nstatus\nor\nStatus\nAmendment\nFor\nthe\nstatus\nAmendment\nthere\ncan\nbe\n3\ntypes\nof\nguarantor:\n●\nCitizen\n●\nGCC\ncountries\ncitizen\n●\nForeign\ncitizen\nFor\nmodifying\nthe\nstatus\nof\na\ncitizen’s\nfamily\nmember\nor,\nmodifying\nthe\nstatus\nof\nfamily\nmember\nof\na\nGCC\ncitizen,\nor\nmodifying\nstatus\nof\nforeigner’s\nfamily\nmembers,\nthe\nfollowing\ndocuments\nhave\nto\nbe\nsubmitted:\n●\nBefore\n60\ndays\nof\nthe\nexpiration\ndate\nthey\nneed\nto\nsubmit\na\ncopy\nof\nthe\nsponsored\

Document(page_content='Service:\nEdit\nstatus\nor\nStatus\nAmendment\nFor\nthe\nstatus\nAmendment\nthere\ncan\nbe\n3\ntypes\nof\nguarantor:\n●\nCitizen\n●\nGCC\ncountries\ncitizen\n●\nForeign\ncitizen\nFor\nmodifying\nthe\nstatus\nof\na\ncitizen’s\nfamily\nmember\nor,\nmodifying\nthe\nstatus\nof\nfamily\nmember\nof\na\nGCC\ncitizen,\nor\nmodifying\nstatus\nof\nforeigner’s\nfamily\nmembers,\nthe\nfollowing\ndocuments\nhave\nto\nbe\nsubmitted:\n●\nBefore\n60\ndays\nof\nthe\nexpiration\ndate\nthey\nneed\nto\nsubmit\na\ncopy\nof\nthe\nsponsored\nperson\nand\nthe\nsponsor,\na\ncopy\nof\nthe\nsponsors\nresidence\npermit,\na\ncopy\nof\nthe\ncancelled\nentry\npermit\nor\nresidence\npermit\nand\nthe\nnew\nentry\npermit.\n●\nAfter\nthe\nexpiration\ndate\nthe\nfines\nstipulated\nwill\nbe\napplied\nand\nthey\nwould\nneed\nto\nsubmit\na\ncopy\nof\nthe\nsponsored\nperson\nand\nthe\nsponsor,\na\ncopy\nof\nthe\nsponsors\nresidence\npermit,\na\ncopy\nof\nthe\ncancelled\nentry\npermit\nor\nresidence\npe